In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 데이터 경로 설정
base_dir = "/content/drive/MyDrive/business_license_dataset"


In [ ]:
pip install transformers datasets torch torchvision


In [ ]:
import torch
import torch.nn as nn
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from transformers import ViTForImageClassification
from torch.optim import AdamW  # PyTorch 공식 AdamW 사용

#  데이터 경로 설정
base_dir = "/content/drive/MyDrive/business_license_dataset"
clear_dir = os.path.join(base_dir, "clearimage")
blur_dir = os.path.join(base_dir, "blurrimage")

#  이미지 확장자 필터링 함수
def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))

#  이미지 파일만 가져오기 (.DS_Store 같은 비이미지 파일 제거)
clear_images = [os.path.join(clear_dir, img) for img in os.listdir(clear_dir) if is_image_file(img)]
blur_images = [os.path.join(blur_dir, img) for img in os.listdir(blur_dir) if is_image_file(img)]

#  이미지 경로 & 라벨 설정 (Blur = 0, Clear = 1)
image_paths = clear_images + blur_images
labels = [1] * len(clear_images) + [0] * len(blur_images)  # 1 = Clear, 0 = Blur

#  데이터 셔플
import numpy as np
data = list(zip(image_paths, labels))
np.random.shuffle(data)
image_paths, labels = zip(*data)

#  데이터 분할 (80% Train, 20% Validation)
split_idx = int(len(image_paths) * 0.8)
train_paths, val_paths = image_paths[:split_idx], image_paths[split_idx:]
train_labels, val_labels = labels[:split_idx], labels[split_idx:]

#  ViT Feature Extractor (이미지 전처리용)
from transformers import ViTFeatureExtractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

#  이미지 변환 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ViT 입력 크기
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

#  이미지 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths
        self.labels = labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # 이미지 불러오기 및 변환
        image = Image.open(img_path).convert("RGB")
        image = transform(image)

        return image, torch.tensor(label, dtype=torch.long)

#  DataLoader 설정
train_dataset = ImageDataset(train_paths, train_labels)
val_dataset = ImageDataset(val_paths, val_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

#  ViT 모델 로드 (크기 불일치 해결)
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2,  # Blur vs Clear (2개 클래스)
    ignore_mismatched_sizes=True  # 크기 불일치 문제 해결
)

#  GPU 사용 가능하면 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#  손실 함수 & 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=3e-5)  # PyTorch 공식 AdamW 사용

#  학습 함수
def train_model(model, train_loader, val_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        total_loss, correct = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits  # ViT 출력
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

        acc = correct / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Train Acc: {acc:.4f}")

        # 📌 검증 단계
        model.eval()
        correct = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).logits
                correct += (outputs.argmax(1) == labels).sum().item()

        val_acc = correct / len(val_loader.dataset)
        print(f"Validation Accuracy: {val_acc:.4f}")

#  모델 학습 시작 (5 Epoch)
train_model(model, train_loader, val_loader, epochs=5)


In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image

#  확률까지 출력하는 예측 함수
def predict_image(img_path, model):
    image = Image.open(img_path).convert("RGB")  # 이미지 로드
    image = transform(image).unsqueeze(0).to(device)  # 전처리 + 배치 차원 추가

    # 예측 수행
    model.eval()
    with torch.no_grad():
        output = model(image).logits
        probabilities = F.softmax(output, dim=1)  # 확률 변환
        pred = torch.argmax(probabilities).item()
        confidence = probabilities[0, pred].item()  # 예측한 클래스의 확률 값

    class_name = "Clear" if pred == 1 else "Blur"
    return class_name, confidence  # 예측된 클래스 + 확률 값 반환

#  예측할 이미지 설정
test_img_path = "/content/drive/MyDrive/business_license_dataset/clearimage/clear34.png"

#  예측 실행
pred_class, pred_confidence = predict_image(test_img_path, model)
print(f"Prediction: {pred_class} ({pred_confidence * 100:.2f}%)")


In [ ]:
# 모델 전체 저장
model_save_path = "/content/drive/MyDrive/blurrcleardistinguishmodelreal.pth"
torch.save(model, model_save_path)
print(f"모델이 저장되었습니다: {model_save_path}")


In [ ]:
import base64

with open("/content/drive/MyDrive/business_license_dataset/clearimage/clear1.png", "rb") as image_file:
    base64_string = base64.b64encode(image_file.read()).decode('utf-8')
    print(base64_string)